# Tugas Besar I
--------------------------
Kelompok:

- Diki Ardian Wirasandi (13515092)
- Irfan Ariq (13515112)
- Pratamamia Agung Prihatmaja (13515142)

In [2]:
from sklearn import datasets
import math
import numpy as np
from scipy import stats
import time

In [2]:
from scipy import stats

def purity(cluster_pred, label):
    outlier = False
    size_data = len(cluster_pred)
    
    data_per_cluster = [[] for i in range(len(set(cluster_pred)))]
    for i,x in enumerate(cluster_pred):
        if x == -1:
            outlier = True
        data_per_cluster[x].append(label[i])

    sum = 0
    for clust in data_per_cluster:
        sum += stats.mode(clust)[1][0]
    
    if outlier:
        sum -= stats.mode(clust)[1][0]
        size_data -= len(clust)

    return sum/size_data

# Agglomerative Clustering
------------------------------------------
*Agglomerative clustering* merupakan suatu teknik dalam melakukan *clustering* yang menggunakan pendekatan *hierarchical*. Ide utama dari teknik ini adalah dengan melakukan penggabungan dua buah *cluster* dengan jarak terdekat pada setiap iterasi hingga diperoleh banyak *cluster* sesuai yang dikehendaki.

Dalam notasi *pseudocode*, algoritma *agglomerative clustering* adalah sebagai berikut.

```
WHILE (length(clusters_now) > nb_target_cluster) DO
    pair_merged = get_shortest_distance_pair(clusters_now)
    merge_cluster(pair_merged)
    update_cluster(clusters_now)
```

Terdapat beberapa pendekatan dalam menentukan pasangan *cluster* dengan jarak terdekat. Pendekatan tersebut adalah sebagai berikut.

1. ***Complete linkage***. Dilakukan dengan menghitung jarak antara dua titik terjauh pada kedua *cluster*.
![complete linkage](img/complete.png "Complete linkage")

2. ***Single linkage***. Dilakukan dengan menghitung jarak antara dua titik terdekat pada kedua *cluster*.
![single linkage](img/single.png "Single linkage")

3. ***Average linkage***. Dilakukan dengan menghitung jarak rata-rata antara semua pasangan titik pada kedua cluster.
![average linkage](img/average.png "Average linkage")

4. ***Average-group linkage***. Dilakukan dengan menghitung jarak antara *centroid* dari kedua *cluster*.
![average group linkage](img/average_group.png "Average-group linkage")

Teknik penghitungan jarak juga beragam. Dalam implementasi ini, diterapkan jarak *manhattan*, *euclidean*, dan *cosine*.

## Class

In [3]:
import numpy as np
import math

class AgglomerativeClustering:
    '''
    Kelas untuk mengakomodasi nilai metode agglomerative clustering
    '''
    
    # Nilai default parameter
    n_clusters = 2
    linkage = 'complete'
    metrics = 'euclidean'
    
    available_metrics = ['euclidean', 'manhattan', 'cosine']
    available_linkage = ['complete', 'single', 'average_group', 'average']
    
    def __init__(self, n_clusters=n_clusters, linkage=linkage, metrics=metrics):
        '''
        Inisiasi kelas. Parameter yang dibutuhkan untuk setiap kelas diinisiasi atau diisi dengan nilai default 
        '''
        if n_clusters <= 0:
            raise Exception('n_clusters must be higher than 0')
        if metrics not in self.available_metrics:
            raise Exception('No metrics \'' + str(metrics) + '\'. Available metrics '+ str(self.available_metrics))
        if linkage not in self.available_linkage:
            raise Exception('No linkage \'' + str(linkage) + '\'. Available linkage '+ str(self.available_linkage))
        self.metrics = metrics
        self.n_clusters = n_clusters
        self.linkage = linkage
        
    def __euclidean_distance(self, data1, data2):
        '''
        Fungsi untuk menghitung euclidean distance di antara dua vector dengan panjang yang sama
        '''
        sum = 0
        if (len(data1) == len(data2)):
            for x1, x2 in zip(data1, data2):
                sum += (x1 - x2)**2
            dist = math.sqrt(sum)
            return dist
        else:
            raise Exception('Length doesn\'t match')

    def __manhattan_distance(self, data1, data2):
        '''
        Fungsi untuk menghitung manhattan distance di antara dua vector dengan panjang yang sama
        '''
        sum = 0
        if (len(data1) == len(data2)):
            for x1, x2 in zip(data1, data2):
                sum += abs(x1 - x2)
            return sum
        else:
            raise Exception('Length doesn\'t match')
            
    def __cosine_distance(self, data1, data2):
        '''
        Fungsi untuk menghitung cosine distance di antara dua vector dengan panjang yang sama
        '''
        return np.dot(data1, data2) / (np.linalg.norm(data1) * np.linalg.norm(data2))

    def __get_distance(self, data1, data2, metrics):
        '''
        Fungsi untuk menghitung jarak dua vector dengan metric pengukuran jarak yang telah ditentukan
        '''
        if (metrics == 'euclidean'):
            dist = self.__euclidean_distance(data1, data2)
        elif (metrics == 'manhattan'):
            dist = self.__manhattan_distance(data1, data2)
        elif (metrics == 'cosine'):
            dist = self.__cosine_distance(data1, data2)
        else:
            raise Exception('Metrics not defined')
        return dist
    
    def __complete_linkage(self, cluster1, cluster2, dist_matrix):
        '''
        Fungsi untuk menghitung jarak antara dua cluster dengan pendekatan complete linkage
        '''
        max_dist = 0
        for v1 in cluster1:
            for v2 in cluster2:
                if (max_dist < dist_matrix[v1][v2]):
                    max_dist = dist_matrix[v1][v2]
        return max_dist

    def __single_linkage(self, cluster1, cluster2, dist_matrix):
        '''
        Fungsi untuk menghitung jarak antara dua cluster dengan pendekatan single linkage
        '''
        min_dist = None
        for v1 in cluster1:
            for v2 in cluster2:
                if (min_dist is None) or (min_dist > dist_matrix[v1][v2]):
                    min_dist = dist_matrix[v1][v2]
        return min_dist

    def __average_linkage(self, cluster1, cluster2, dist_matrix):
        '''
        Fungsi untuk menghitung jarak antara dua cluster dengan pendekatan average linkage
        '''
        sum_dist = 0
        count_dist = 0
        for v1 in cluster1:
            for v2 in cluster2:
                sum_dist += dist_matrix[v1][v2]
                count_dist += 1
        return float(sum_dist)/float(count_dist)

    def __group_average_linkage(self, cluster1, cluster2, data, distance):
        '''
        Fungsi untuk menghitung jarak antara dua cluster dengan pendekatan average group linkage
        '''
        data1 = [data[i] for i in cluster1]
        data2 = [data[i] for i in cluster2]

        avg1 = np.mean(data1, axis = 0)
        avg2 = np.mean(data2, axis = 0)

        return self.__get_distance(avg1, avg2, distance)
    
    def __calculate_distance_matrix(self, data, metrics):
        '''
        Fungsi untuk menghitung distance matrix untuk semua pasangan vector di dalam data
        '''
        dist_matrix = []
        for idx1, data1 in enumerate(data):
            curr_dist_matrix = []
            for idx2, data2 in enumerate(data):
                if (idx1 > idx2):
                    curr_dist_matrix.append(dist_matrix[idx2][idx1])
                else:
                    dist = self.__get_distance(data1, data2, metrics)
                    curr_dist_matrix.append(dist)
            dist_matrix.append(curr_dist_matrix)
        return dist_matrix
        
    def fit_predict(self, data):
        '''
        Fungsi untuk melakukan clustering secara agglomerative
        '''
        
        # preprocessing distance matrix
        if (self.linkage != 'average_group'):
            dist_matrix = self.__calculate_distance_matrix(data, self.metrics)
        # inisiasi cluster dengan satu elemen per cluster awal 
        clusters = [[i] for i, c in enumerate(data)]

        # melakukan iterasi hingga diperoleh jumlah cluster sesuai yang dikehendaki
        while(len(clusters) > self.n_clusters):
            min_dist = None
            merge_pair = (0, 0)
            
            # mencari cluster dengan jarak terdekat untuk di-merge
            for idx1, c1 in enumerate(clusters):
                for idx2, c2 in enumerate(clusters[(idx1 + 1) :]):
                    if (self.linkage == 'single'):
                        dist = self.__single_linkage(c1, c2, dist_matrix)
                    elif (self.linkage == 'complete'):
                        dist = self.__complete_linkage(c1, c2, dist_matrix)
                    elif (self.linkage == 'average'):
                        dist = self.__average_linkage(c1, c2, dist_matrix)
                    elif (self.linkage == 'average_group'):
                        dist = self.__group_average_linkage(c1, c2, data, self.metrics)
                    else:
                        raise Exception('Linkage not defined')
                    if (min_dist == None) or (dist < min_dist):
                        min_dist = dist
                        merge_pair = (idx1, idx1 + 1 + idx2)
            
            # merge pasangan cluster dengan jarak terdekat
            result_cluster = []
            for idx, c in enumerate(clusters):
                if idx not in merge_pair:
                    result_cluster.append(c)

            result_cluster.append(clusters[merge_pair[0]] + clusters[merge_pair[1]])

            clusters = result_cluster

        # menampilkan hasil clustering
        result_per_item = np.full(len(data), 0)
        for idx, clust in enumerate(clusters):
            result_per_item[clust] = idx

        return result_per_item
    
    '''
    Getter untuk parameter
    '''
    def get_n_cluster(self):
        return self.n_clusters
    
    def get_linkage(self):
        return self.linkage
    
    def get_metrics(self):
        return self.metrics

## Experiments
Berikut adalah eksperimen yang dilakukan untuk melakukan *clustering* terhadap *dataset* iris. Digunakan *euclidean distance* dan berbagai variasi perhitungan *linkage* antara dua cluster untuk menerapkan metode *agglomerative clustering*.

In [4]:
iris = datasets.load_iris()
data = iris.data
label = iris.target

#### Clustering dengan Complete Linkage

Pada percobaan ini, digunakan pendekatan *complete linkage* untuk menghitung jarak antar pasangan cluster. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [5]:
aglo = AgglomerativeClustering(n_clusters=3, linkage='complete', metrics='euclidean')
start = time.time()
pred = aglo.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.8769066333770752 s ----
Cluster prediction:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 0 2 0 2 0 2 0 0 0 0 2 0 2 0 0 2 0 2 0 2 2
 2 2 2 2 2 0 0 0 0 2 0 2 2 2 0 0 0 2 0 0 0 0 0 2 0 0 2 2 2 2 2 2 0 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
Purity: 0.84


#### Clustering dengan Single Linkage

Pada percobaan ini, digunakan pendekatan *single linkage* untuk menghitung jarak antar pasangan cluster. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [6]:
aglo = AgglomerativeClustering(n_clusters=3, linkage='single', metrics='euclidean')
start = time.time()
pred = aglo.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.7279071807861328 s ----
Cluster prediction:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
Purity: 0.68


#### Clustering dengan Average Linkage

Pada percobaan ini, digunakan pendekatan *complete linkage* untuk menghitung jarak antar pasangan cluster. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [7]:
aglo = AgglomerativeClustering(n_clusters=3, linkage='average', metrics='euclidean')
start = time.time()
pred = aglo.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 1.2760462760925293 s ----
Cluster prediction:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 1 1 1 1 2 1 1 1 1
 1 1 2 2 1 1 1 1 2 1 2 1 2 1 1 2 2 1 1 1 1 1 2 1 1 1 1 2 1 1 1 2 1 1 1 2 1
 1 2]
Purity: 0.9066666666666666


#### Clustering dengan Average-group Linkage

Pada percobaan ini, digunakan pendekatan *average-group linkage* untuk menghitung jarak antar pasangan cluster. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [8]:
aglo = AgglomerativeClustering(n_clusters=3, linkage='average_group', metrics='euclidean')
start = time.time()
pred = aglo.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 44.81160497665405 s ----
Cluster prediction:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0 0 0 0 2 0 0 0 0
 0 0 2 2 0 0 0 0 2 0 2 0 2 0 0 2 2 0 0 0 0 0 2 0 0 0 0 2 0 0 0 2 0 0 0 2 0
 0 2]
Purity: 0.9066666666666666


## Hasil dan Analisis

Dari keempat eksperimen di atas, dapat ditarik kesimpulan bahwa untuk dataset iris, metode *agglomerative* dapat diterapkan untuk melakukan *clustering*. Penggunaan teknik *average linkage* dan *average-group linkage* untuk menghitung jarak antara dua cluster menghasilkan *cluster* dengan *purity* tertinggi, yaitu 0.9067. Namun, *average-group linkage* membutuhkan waktu eksekusi yang lebih lama dibandingkan *average linkage*. Hal ini dikarenakan teknik *average-group linkage* tidak dapat menggunakan hasil *preprocessing distance matrix* sehingga perlu dilakukan komputasi ulang dalam menghitung jarak antara dua *cluster* .

# DBSCAN
----------------------------

DBSCAN merupakan salah satu algoritma clustering yang mengelompokkan data berdasarkan kedekatannya dengan data lain. Data yang dianggap dekat akan dijadikan satu kelompok. Data dianggap dekat dan disebut bertetangga dengan data lainnya apabila jaraknya kurang dari smaa dengan nilai tertentu. Nilai tersebut disebut `epsilon`.

Pada DBSCAN satu *instance* data dapat dikategorikan menjadi `core_point`, `border_point`, atau `noise_data`/outlier. Sebuah data disebut `core_point` apabila memiliki jumlah tetangganya lebih dari sama dengan nilai tertentu. Nilai tersebut disebut `min_pts`. Sebuah data dikatakan `border_point` apabila jumlah tetangganya tidak lebih dari `min_pts` namun memiliki tetangga yang merupakan `core_point`. Sedangkan `noise_data` atau outlier adalah data yang jumlah tetangganya tidak lebih dari `min_pts` dan tidak bertetangga dengan `border_point`. 

Setiap `core_point` dan tetangganya (baik itu `core_point` atau pun `border_point`) akan menjadi satu cluster yang sama. `noise_data` atau outlier merupakan data yang tidak memiliki cluster. 

Ilustrasi: 
![DBSCAN](img/dbscan.png "Ilustrasi DBSCAN")

Pada gambar diatas, titik yang berwarna merah merupakan `core_point`, titik yang berwarna kuning merupakan `border_point` dan titik yang berwarna biru merupakan `noise_data` atau outlier.

Perhitungan jarak yang dapat digunakan pada implemetasi DBSCAN ini ada dua macam yaitu jarak euclidean dan jarak manhtattan.

Berikut ini merupakan pseudocode dari DBSCAN:

```
DBSCAN(data, eps, min_pts):
    curr_label = 0
    for data_i in data:
        if data_i is core_point and not yet labelled:
            label = curr_label
            cluster(data_i) = label
            neighbour_stack = [neighbour(data_i)]
            while neighbour_stack is not empty:
                neighbour_data_i = neighbour_stack.pop
                if neighbour_data_i not yet labelled:
                    cluster(neighbour_data_i) = label
                    if neighbour_data_i is core point:
                        neighbour_stack,push(neighbour(neighbour_data_i))
           curr_label += 1
```

## Class

In [9]:
import numpy as np 
import math

class tes_DBSCAN:
    
    UNLABELLED_DATA = -1
    
    n_clusters = None
    result = None
        
    metrics = 'euclidean'    
    eps = 0.5
    min_pts = 5
    available_metrics = ['euclidean', 'manhattan']

    def __init__ (self, min_pts=min_pts, eps=eps, metrics=metrics):
        '''
        Inisiasi kelas dengan min_pts dan epsilon
        '''
        if eps <= 0:
            raise Exception('eps must be higher than 0')
        if min_pts <= 0:
            raise Exception('min_pts must be higher than 0')
        if metrics not in self.available_metrics:
            raise Exception('No metrics \'' + str(metrics) + '\'. Available metrics '+ str(self.available_metrics))
            
        self.min_pts = min_pts
        self.eps = eps
        self.metrics=metrics
        
    def __euclidean_distance(self, point_a, point_b):
        '''
        Fungsi untuk menghitung euclidean distance
        '''
        dist = 0
        for a, b in zip(point_a, point_b):
            dist += (a - b) * (a - b)
        return np.sqrt(dist)

    def __manhattan_distance(self, point_a, point_b):
        '''
        Fungsi untuk menghitung manhattan distance 
        '''
        dist = 0
        for a, b in zip(point_a, point_b):
            dist += abs(a - b)
        return dist
    
    def __distance(self, point_a, point_b, metrics=metrics):
        '''
        Fungsi untuk mencari jarak berdasarkan metricsnya
        '''
        if len(point_a) == len(point_b):
            if metrics == 'euclidean':
                return self.__euclidean_distance(point_a, point_b)
            if metrics == 'manhattan':
                return self.__manhattan_distance(point_a, point_b)
        else:
            raise Exception("feature length doesn't same")
    
    def fit_predict(self, data):
        '''
        Fungsi untuk mengelompolkkan data
        '''
        size_data = len(data)
        
        # generate all neighbours 
        neighbours = []
        for i in range(size_data):
            neighbour_i = []
            for j in range(size_data):
                if self.__distance(data[i], data[j], self.metrics) <= self.eps:
                    neighbour_i.append(j)
            neighbours.append(neighbour_i)
        
        # initialize label
        self.result = np.full((size_data), self.UNLABELLED_DATA)
        
        # giving label to data
        curr_label = 0
        for i in range(size_data):
            # if neighbours > min_pts (data_i is core points) and not yet labelled, then give label 
            if len(neighbours[i]) >= self.min_pts and self.result[i] == self.UNLABELLED_DATA: 
                label = curr_label
                # giving label to all neighbours
                neighbours_i = [i]
                while len(neighbours_i) > 0:
                    neigh_i = neighbours_i.pop()
                    # if not yet labelled then give label to data and the neighbours
                    if self.result[neigh_i] == self.UNLABELLED_DATA:
                        self.result[neigh_i] = label
                        # if neigh_i is core point, then give label to the neighbour
                        if len(neighbours[neigh_i]) >= self.min_pts:
                            neighbours_i += neighbours[neigh_i]
                curr_label += 1
        
        self.n_clusters = curr_label           
        return self.result
    
    def get_n_clusters(self):
        if self.n_clusters is None:
            print("No data")
        else:
            return self.n_clusters
    
    def get_epsilon(self):
        return self.eps

    def get_metrics(self):
        return self.metrics
    
    def get_min_pts(self):
        return self.min_pts
    

## Experiments

Berikut ini merupakan hasil eksperimen implementasi DBSCAN untuk clustering data iris menggunakan *euclidean disntance*.

In [10]:
from sklearn import datasets

iris = datasets.load_iris()
data = iris.data
label = iris.target

In [11]:
import time

dbscan = tes_DBSCAN(eps=0.5, min_pts=4, metrics='euclidean')

start = time.time()

pred = dbscan.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("---- Cluster: {} ----".format(dbscan.get_n_clusters()))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.3169534206390381 s ----
---- Cluster: 3 ----
Cluster prediction:
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 -1  0  0  0  0  0  0
  0  0  1  1  1  1  1  1  1  2  1  1  2  1  1  1  1  1  1  1 -1  1  1  1
  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 -1  1  1  1  1  1  2  1  1
  1  1  2  1  1  1  1  1  1 -1 -1  1 -1 -1  1  1  1  1  1  1  1 -1 -1  1
  1  1 -1  1  1  1  1  1  1  1  1 -1  1  1 -1 -1  1  1  1  1  1  1  1  1
  1  1  1  1  1  1]
Purity: 0.708029197080292


In [12]:
dbscan = tes_DBSCAN(eps=1, min_pts=3, metrics='euclidean')

start = time.time()

pred = dbscan.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("---- Cluster: {} ----".format(dbscan.get_n_clusters()))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.2816331386566162 s ----
---- Cluster: 2 ----
Cluster prediction:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1]
Purity: 0.6666666666666666


## Hasil Eksperimen

Dari kedua hasil eksperimen diatas, dapat dilihat bahwa DBSCAN mampu mengelompokkan data iris dalam waktu sekitar 0.2 - 0.3  detik. Namun kedua eksperimen menghasilkan nilai purity yang berbeda, hal ini dikarenakan perbedaan nilai `epsilon` dan `min_pts`. Dengan nilai `epsilon` 0.5 dan `min_pts` 4, DBSCAN menghasilkan 3 kluster dan beberapa data yang dianggap outlier. Sedangkan dengan nilai `epsilon` 1 dan `min_pts` 3, DBSCAN menghasilkan 2 kluster dan tanpa ada data yang dianggap outlier. Hal ini menunjukkan bahwa DBSCAN akan sangat bergantung terhadap kedua nilai tersebut dan ini merupakan tantangan dalam menggunakan DBSCAN.

# KMeans
------------------------------------------
*KMeans* merupakan suatu teknik dalam melakukan *clustering* yang menggunakan pendekatan *partitioning*. Ide utama dari teknik ini adalah dengan mengelompokkan data ke K *cluster*. Penentuan *cluster* mana untuk data tertentu yaitu dengan cara menghitung jarak data tersebut dengan representasi dari *cluster* (*centroid*). Representasi dari *cluster* berupa rata-rata (*means*) dari data-data anggota *cluster* tersebut. Penghitungan jarak dengan menggunakan *euclidean distance*.

![kmeans](img/kmeans.png "KMeans Clustering")

Dalam notasi *pseudocode*, algoritma *KMeans* adalah sebagai berikut.

```
choose_initial_centroid()
WHILE (any_change_in_cluster) DO
    FOREACH object
        assign_to_cluster(object)
    FOREACH cluster
        update_cluster_means(cluster)
```


## Implementation Class

In [13]:
import numpy as np
import math
import random

class KMeans:
    '''
    Kelas untuk mengakomodasi nilai metode KMeans clustering
    '''
    # Nilai default parameter
    n_clusters = 2
    init = 'random'
    max_iter = 300
    init_val = []
    
    available_init = ['random', 'manual']
    
    def __init__(self, n_clusters=n_clusters, init=init, init_val=init_val, max_iter=max_iter):
        '''
        Inisiasi kelas. Parameter yang dibutuhkan untuk setiap kelas diinisiasi atau diisi dengan nilai default 
        '''    
        if n_clusters <= 0:
            raise Exception('n_clusters must be higher than 0')
        if init not in self.available_init:
            raise Exception('No init method \'' + str(init) + '\'. Available init methods'+ str(self.available_init))
        if (init == 'manual' and len(init_val) != n_clusters):
            raise Exception('init_val length doesn\'t match with n_clusters '+ str(n_clusters))
        self.n_clusters = n_clusters
        self.init = init
        self.max_iter = max_iter
        self.init_val = init_val
    
    def __is_in_array(self, data, arr):
        '''
        Fungsi helper untuk menegecek apakah data berupa array berada pada arr berupa array of array
        '''
        is_exist = False
        arr_idx = 0
        while (not is_exist and arr_idx < len(arr)):
            is_data_equal = True
            data_idx = 0
            while (is_data_equal and data_idx < len(data)):
                if (data[data_idx] != arr[arr_idx][data_idx]):
                    is_data_equal = False
                else:
                    data_idx += 1
            if is_data_equal:
                is_exist = True
            else:
                arr_idx += 1
        return is_exist
        
    def __euclidean_distance(self, data1, data2):
        '''
        Fungsi untuk menghitung euclidean distance di antara dua vector dengan panjang yang sama
        '''
        sum = 0
        if (len(data1) == len(data2)):
            for x1, x2 in zip(data1, data2):
                sum += (x1 - x2)**2
            dist = math.sqrt(sum)
            return dist
        else:
            raise Exception('Length doesn\'t match')
            
    def __get_distance(self, data1, data2):
        '''
        Fungsi untuk menghitung jarak dua vector
        '''
        return self.__euclidean_distance(data1, data2)
        
    def __calculate_distance_matrix(self, data, centroids):
        '''
        Fungsi untuk menghitung distance matrix untuk semua data dengan centroid
        '''
        dist_matrix = []        
        for i in range(len(centroids)):
            dist_curr_centroid = []
            for j in range(len(data)):
                dist = self.__get_distance(centroids[i], data[j])
                dist_curr_centroid.append(dist)
            dist_matrix.append(dist_curr_centroid)
        
        return dist_matrix
    
    def __assign_data_to_cluster(self, dist_matrix):
        '''
        Fungsi untuk mengelompokan data berdasarkan jarak yang diketahui
        '''
        cluster_of_data = []
        for j in range(len(dist_matrix[0])):
            cluster = 0
            min_distance = dist_matrix[0][j]
            for i in range(1,len(dist_matrix)):
                if (dist_matrix[i][j] < min_distance):
                    cluster = i
                    min_distance = dist_matrix[i][j]
            cluster_of_data.append(cluster)
        return cluster_of_data
        
    def __get_centroids(self, data, cluster_of_data):
        '''
        Fungsi untuk menghitung centroid baru
        '''
        centroids = []
        data_per_cluster = []
        # inisiasi
        for n in range(self.n_clusters):
            data_per_cluster.append([])
        # masukkan data ke array tiap cluster
        for idx, data_cluster in enumerate(cluster_of_data):
            data_per_cluster[data_cluster].append(data[idx])
        # hitung means
        for n in range(self.n_clusters):
            if len(data_per_cluster[n]) > 0:
                means_cluster = []
                for column in range(len(data_per_cluster[n][0])):
                    sum_column = 0
                    for d in data_per_cluster[n]:
                        sum_column += d[column]
                    means_column = sum_column / len(data_per_cluster[n])
                    means_cluster.append(means_column)
                centroids.append(means_cluster)
        return centroids
        
    def fit_predict(self, data):
        '''
        Fungsi untuk melakukan clustering secara KMeans
        '''
        
        cluster_of_data = []
        
        # initiate centroid
        centroids = []
        if (self.init == 'random'):
            # cek keunikan data
            unique_data_idx = []
            unique_data = []
            i = 0
            while (len(unique_data_idx) < self.n_clusters) and (i < len(data)):
                if not self.__is_in_array(data[i], unique_data):
                    unique_data_idx.append(i)
                    unique_data.append(data[i])
                i += 1
                
            if (len(unique_data_idx) < self.n_clusters):
                # jika keunikan data kurang dari n_clusters
                for u in unique_data_idx:
                    curr_centroid = np.copy(data[u])
                    centroids.append(curr_centroid)
                for i in range(self.n_clusters - len(unique_data_idx)):
                    rand_idx = random.randint(-1,len(data)-1)
                    # cek apakah sudah terpilih atau belum
                    while (rand_idx in unique_data_idx):
                        rand_idx = random.randint(-1,len(data)-1)
                    curr_centroid = np.copy(data[rand_idx])
                    centroids.append(curr_centroid)
            else:
                for i in range(self.n_clusters):
                    rand_idx = random.randint(-1,len(data)-1)
                    curr_centroid = np.copy(data[rand_idx])
                    # cek apakah sudah terpilih atau belum
                    while (self.__is_in_array(curr_centroid, centroids)):
                        rand_idx = random.randint(-1,len(data)-1)
                        curr_centroid = np.copy(data[rand_idx])
                    centroids.append(curr_centroid)
        else:
            # self.init == 'manual'
            centroids = self.init_val
        
        iteration = 1
        is_convergen = False
        while (not is_convergen and iteration <= self.max_iter):
            # calculate distance all data to all centroid
            dist_matrix = self.__calculate_distance_matrix(data, centroids)
            # assign all data to cluster
            new_cluster_of_data = self.__assign_data_to_cluster(dist_matrix)
            # convergency checking
            is_convergen = np.array_equal(cluster_of_data, new_cluster_of_data)
            # for next iteration
            if not is_convergen:
                cluster_of_data = np.copy(new_cluster_of_data)
                centroids = self.__get_centroids(data, cluster_of_data)
                iteration += 1
    
        return cluster_of_data

## Experiments

Berikut ini merupakan hasil eksperimen implementasi KMeans untuk clustering data iris

#### Clustering dengan k=3

Pada percobaan ini, dipilih nilai k=3 dan dilakukan percobaan sebanyak 2 kali. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [14]:
kmeans = KMeans(n_clusters=3)
start = time.time()
pred = kmeans.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.0557103157043457 s ----
Cluster prediction:
[2 1 1 1 2 2 1 2 1 1 2 1 1 1 2 2 2 2 2 2 2 2 2 2 1 1 2 2 2 1 1 2 2 2 1 2 2
 1 1 2 2 1 1 2 2 1 2 1 2 2 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0]
Purity: 0.6666666666666666


In [15]:
kmeans = KMeans(n_clusters=3)
start = time.time()
pred = kmeans.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.04532933235168457 s ----
Cluster prediction:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 2 2 1 2 2 2 2
 2 2 1 1 2 2 2 2 1 2 1 2 1 2 2 1 1 2 2 2 2 2 1 2 2 2 2 1 2 2 2 1 2 2 2 1 2
 2 1]
Purity: 0.8866666666666667


#### Clustering dengan k=4

Pada percobaan ini, dipilih nilai k=4 dan dilakukan percobaan sebanyak 2 kali. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [16]:
kmeans = KMeans(n_clusters=4)
start = time.time()
pred = kmeans.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.07248306274414062 s ----
Cluster prediction:
[3 1 1 1 3 3 1 3 1 1 3 1 1 1 3 3 3 3 3 3 3 3 1 3 1 1 3 3 3 1 1 3 3 3 1 1 3
 1 1 3 3 1 1 3 3 1 3 1 3 1 2 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 2 2 2 2 0 2 2 2 2
 2 2 0 0 2 2 2 2 0 2 0 2 0 2 2 0 0 2 2 2 2 2 0 2 2 2 2 0 2 2 2 0 2 2 2 0 2
 2 0]
Purity: 0.8866666666666667


In [17]:
kmeans = KMeans(n_clusters=4)
start = time.time()
pred = kmeans.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.030618667602539062 s ----
Cluster prediction:
[0 2 2 2 0 0 2 2 2 2 0 2 2 2 0 0 0 0 0 0 0 0 2 0 2 2 0 0 0 2 2 0 0 0 2 2 0
 2 2 0 0 2 2 0 0 2 0 2 0 2 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 3 1 3 3 3 3 1 3 3 3 3
 3 3 1 1 3 3 3 3 1 3 1 3 1 3 3 1 1 3 3 3 3 3 1 3 3 3 3 1 3 3 3 1 3 3 3 1 3
 3 1]
Purity: 0.8933333333333333


## Hasil dan Analisis

Dari keempat eksperimen di atas, dapat ditarik kesimpulan bahwa untuk dataset iris, metode *KMeans* dapat diterapkan untuk melakukan *clustering*. Pemilihan nilai k=4 menghasilkan *cluster* dengan *purity* tertinggi, yaitu 0.893. Baik nilai k=3 atau k=4 membutuhkan waktu eksekusi yang relatif sama 0.01 detik. Terjadi perbedaan antara percobaan pertama dan kedua pada k=3, hal ini dikarenakan inisiasi centroid yang random membuat hasil yang tidak sama. Meskipun pada k=4, percobaan pertama dan kedua menghasilkan *purity* yang sama, namun untuk percobaan ketiga dan selanjutnya belum tentu hasil yang sama akan didapatkan karena penggunaan teknik *kmeans* ini sangat sensitif terhadap inisiasi centroidnya. Untuk itu, sebaiknya dilakukan percobaan berulang-ulang hingga mendapatkan hasil yang terbaik.

# KMedoid (PAM)
------------------------------------------
*KMedoid* merupakan suatu teknik dalam melakukan *clustering* yang menggunakan pendekatan *partitioning*. Ide utama dari teknik ini adalah dengan mengelompokkan data ke K *cluster*. Penentuan *cluster* mana untuk data tertentu yaitu dengan cara menghitung jarak data tersebut dengan representasi dari *cluster* (*centroid*). Representasi dari *cluster* berupa sebuah data (*medoid*) dari data-data anggota *cluster* tersebut. Penghitungan jarak dengan menggunakan *manhattan distance*. *Medoid* akan di-update setiap iterasi dengan cara pemilihan secara random pada cluster tertentu, kemudian dihitung perubahan *error*. *Error* berupa *absolute error*, jika perubahan *error* < 0, maka iterasi akan berlanjut dan *medoid* di-update. 

![kmedoid](img/kmedoid.png "KMedoid Clustering")

Dalam notasi *pseudocode*, algoritma *KMedoid* adalah sebagai berikut.

```
choose_initial_centroid()
REPEAT
    FOREACH object
        assign_to_cluster(object)
    choose_new_centroid()
    calculate_total_error()
    if (total_error < 0)
        swap(old_centroid, new_centroid)
UNTIL no_changes
```


## Implementation Class

In [18]:
import numpy as np
import math
import random

class KMedoid:
    '''
    Kelas untuk mengakomodasi nilai metode KMedoid clustering
    '''
    # Nilai default parameter
    n_clusters = 2
    init = 'random'
    max_iter = 300
    init_val = []
    randomize_cluster = 0
    choosen_cluster = []
    
    available_init = ['random', 'manual']
    
    def __init__(self, n_clusters=n_clusters, init=init, init_val=init_val,
                 max_iter=max_iter, randomize_cluster=randomize_cluster):
        '''
        Inisiasi kelas. Parameter yang dibutuhkan untuk setiap kelas diinisiasi atau diisi dengan nilai default 
        '''
        if n_clusters <= 0:
            raise Exception('n_clusters must be higher than 0')
        if init not in self.available_init:
            raise Exception('No init method \'' + str(init) + '\'. Available init methods'+ str(self.available_init))
        if (init == 'manual' and len(init_val) != n_clusters):
            raise Exception('init_val length doesn\'t match with n_clusters '+ str(n_clusters))
        if (n_clusters-1 < randomize_cluster) or (randomize_cluster < 0):
            raise Exception('randomize_cluster must be between 0 and n_clusters-1')
        self.n_clusters = n_clusters
        self.init = init
        self.max_iter = max_iter
        self.init_val = init_val
        self.randomize_cluster = randomize_cluster
    
    def __is_in_array(self, data, arr):
        '''
        Fungsi helper untuk menegecek apakah data berupa array berada pada arr berupa array of array
        '''
        is_exist = False
        arr_idx = 0
        while (not is_exist and arr_idx < len(arr)):
            is_data_equal = True
            data_idx = 0
            while (is_data_equal and data_idx < len(data)):
                if (data[data_idx] != arr[arr_idx][data_idx]):
                    is_data_equal = False
                else:
                    data_idx += 1
            if is_data_equal:
                is_exist = True
            else:
                arr_idx += 1
        return is_exist
        
    def __manhattan_distance(self, data1, data2):
        '''
        Fungsi untuk menghitung manhattan distance di antara dua vector dengan panjang yang sama
        '''
        sum = 0
        if (len(data1) == len(data2)):
            for x1, x2 in zip(data1, data2):
                sum += abs(x1 - x2)
            return sum
        else:
            raise Exception('Length doesn\'t match')
            
    def __get_distance(self, data1, data2):
        '''
        Fungsi untuk menghitung jarak dua vector
        '''
        return self.__manhattan_distance(data1, data2)
        
    def __calculate_distance_matrix(self, data, centroids):
        '''
        Fungsi untuk menghitung distance matrix untuk semua data dengan centroid
        '''
        dist_matrix = []        
        for i in range(len(centroids)):
            dist_curr_centroid = []
            for j in range(len(data)):
                dist = self.__get_distance(centroids[i], data[j])
                dist_curr_centroid.append(dist)
            dist_matrix.append(dist_curr_centroid)
        
        return dist_matrix
    
    def __assign_data_to_cluster(self, dist_matrix):
        '''
        Fungsi untuk mengelompokan data berdasarkan jarak yang diketahui
        '''
        cluster_of_data = []
        for j in range(len(dist_matrix[0])):
            cluster = 0
            min_distance = dist_matrix[0][j]
            for i in range(1,len(dist_matrix)):
                if (dist_matrix[i][j] < min_distance):
                    cluster = i
                    min_distance = dist_matrix[i][j]
            cluster_of_data.append(cluster)
        return cluster_of_data
    
    def __get_centroids(self, data, cluster_of_data, centroids):
        '''
        Fungsi untuk mendapatkan centroid baru
        '''
        # centroid candidate
        data_of_randomize_cluster = []
        for idx, data_cluster in enumerate(cluster_of_data):
            if data_cluster == self.randomize_cluster:
                data_of_randomize_cluster.append(data[idx])
        
        # choose random
        idx = 0
        stop = False
        while ( not stop and idx < len(data_of_randomize_cluster)
        ):
            new_centroid = np.copy(data_of_randomize_cluster[idx])
            if (
                self.__get_distance(new_centroid, centroids[self.randomize_cluster]) == 0 or 
                self.__is_in_array(new_centroid, self.choosen_cluster)
            ):
                idx += 1
            else:
                stop = True
        
        if not self.__is_in_array(new_centroid, self.choosen_cluster):
            self.choosen_cluster.append(new_centroid)
        
        new_centroids = np.copy(centroids)
        new_centroids[self.randomize_cluster] = new_centroid
        return new_centroids
    
    def __calculate_error(self, data, cluster_of_data, new_cluster_of_data, centroids, new_centroids):
        '''
        Fungsi untuk menghitung total absolute error
        '''
        
        old_error = 0
        new_error = 0
        for n in range(self.n_clusters):
            for idx, val in enumerate(data):
                old_error += self.__get_distance(val, centroids[cluster_of_data[idx]])
                new_error += self.__get_distance(val, new_centroids[new_cluster_of_data[idx]])
        
        return new_error-old_error
        
    def fit_predict(self, data):
        '''
        Fungsi untuk melakukan clustering secara KMedoid
        '''
        
        cluster_of_data = []
        
        # initiate centroid
        centroids = []
        if (self.init == 'random'):
            # cek keunikan data
            unique_data_idx = []
            unique_data = []
            i = 0
            while (len(unique_data_idx) < self.n_clusters) and (i < len(data)):
                if not self.__is_in_array(data[i], unique_data):
                    unique_data_idx.append(i)
                    unique_data.append(data[i])
                i += 1
                
            if (len(unique_data_idx) < self.n_clusters):
                # jika keunikan data kurang dari n_clusters
                for u in unique_data_idx:
                    curr_centroid = np.copy(data[u])
                    centroids.append(curr_centroid)
                for i in range(self.n_clusters - len(unique_data_idx)):
                    rand_idx = random.randint(-1,len(data)-1)
                    # cek apakah sudah terpilih atau belum
                    while (rand_idx in unique_data_idx):
                        rand_idx = random.randint(-1,len(data)-1)
                    curr_centroid = np.copy(data[rand_idx])
                    centroids.append(curr_centroid)
            else:
                for i in range(self.n_clusters):
                    rand_idx = random.randint(-1,len(data)-1)
                    curr_centroid = np.copy(data[rand_idx])
                    # cek apakah sudah terpilih atau belum
                    while (self.__is_in_array(curr_centroid, centroids)):
                        rand_idx = random.randint(-1,len(data)-1)
                        curr_centroid = np.copy(data[rand_idx])
                    centroids.append(curr_centroid)
        else:
            # self.init == 'manual'
            centroids = self.init_val
        
        iteration = 1
        is_convergen = False
        
        while (not is_convergen and iteration <= self.max_iter):
            # calculate distance all data to all centroid
            dist_matrix = self.__calculate_distance_matrix(data, centroids)
            # assign all data to cluster
            cluster_of_data = self.__assign_data_to_cluster(dist_matrix)
            # get new possible centroid
            new_centroids = self.__get_centroids(data, cluster_of_data, centroids)
            # calculate distance all data to all new centroid
            new_dist_matrix = self.__calculate_distance_matrix(data, new_centroids)
            # assign all data to new cluster
            new_cluster_of_data = self.__assign_data_to_cluster(new_dist_matrix)
            # convergency checking
            if (self.__calculate_error(data, cluster_of_data, new_cluster_of_data, centroids, new_centroids) >= 0):
                is_convergen = True
            
            # for next iteration
            if not is_convergen:
                cluster_of_data = np.copy(new_cluster_of_data)
                centroids = np.copy(new_centroids)
                iteration += 1
                
        return np.array(cluster_of_data)

## Experiments

Berikut ini merupakan hasil eksperimen implementasi KMedoid (PAM) untuk clustering data iris

#### Clustering dengan k=4, randomize_cluster=0

Pada percobaan ini, dipilih nilai k=4 dan randomize_cluster=0, artinya cluster yang akan dirandom centroidnya untuk setiap iterasi adalah cluster pertama. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [23]:
kmedoid = KMedoid(n_clusters=4, randomize_cluster=0)
start = time.time()
pred = kmedoid.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.09967494010925293 s ----
Cluster prediction:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 0 0 0 3 0 3 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0
 0 3 3 3 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 2 2 2 2 0 2 2 2 2
 2 2 0 0 2 2 2 2 0 2 0 2 0 2 2 0 0 2 2 2 2 2 0 0 2 2 2 0 2 2 2 0 2 2 2 0 2
 2 0]
Purity: 0.9


#### Clustering dengan k=4, randomize_cluster=1

Pada percobaan ini, dipilih nilai k=4 dan randomize_cluster=1, artinya cluster yang akan dirandom centroidnya untuk setiap iterasi adalah cluster kedua. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [20]:
kmedoid = KMedoid(n_clusters=4, randomize_cluster=1)
start = time.time()
pred = kmedoid.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.0945584774017334 s ----
Cluster prediction:
[3 0 0 0 3 1 3 3 0 0 1 3 0 0 1 1 1 3 1 3 1 3 3 3 3 0 3 3 3 0 0 1 1 1 0 3 1
 0 0 3 3 0 0 3 3 0 3 0 1 3 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
Purity: 0.6666666666666666


#### Clustering dengan k=4, randomize_cluster=2

Pada percobaan ini, dipilih nilai k=4 dan randomize_cluster=2, artinya cluster yang akan dirandom centroidnya untuk setiap iterasi adalah cluster ketiga. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [26]:
kmedoid = KMedoid(n_clusters=4, randomize_cluster=2)
start = time.time()
pred = kmedoid.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))

---- Time taken: 0.06009364128112793 s ----
Cluster prediction:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 3 2 3 2 3 2 3 3 3 3 2 3 2 2 3 2 3 2 3 2 2
 2 2 2 2 2 3 3 3 3 1 2 2 2 2 3 3 3 2 3 3 3 3 3 2 3 3 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 2 2 1 2 1 1 1 2 2 1 1 1 2 1 1 1 1 1 1 1 1 1
 1 1]
Purity: 0.9466666666666667


#### Clustering dengan k=4, randomize_cluster=3

Pada percobaan ini, dipilih nilai k=4 dan randomize_cluster=0, artinya cluster yang akan dirandom centroidnya untuk setiap iterasi adalah cluster keempat. Hasil clustering dan *purity*-nya dapat dilihat pada *output* dari eksekusi *code*.

In [22]:
kmedoid = KMedoid(n_clusters=4, randomize_cluster=3)
start = time.time()
pred = kmedoid.fit_predict(data)

print("---- Time taken: {} s ----".format(time.time() - start))
print("Cluster prediction:")
print(pred)
print("Purity: {}".format(purity(pred, label)))


---- Time taken: 0.05040931701660156 s ----
Cluster prediction:
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 3 3 3 2 0 2 3 2 3 2 2 2 2 0 2 3 2 2 0 2 3 2 0 2
 0 3 3 3 0 2 2 2 2 0 2 0 3 0 2 2 2 0 2 2 2 2 2 0 2 2 3 2 3 3 3 3 0 3 3 3 3
 3 3 0 2 3 3 3 3 0 3 0 3 0 3 3 0 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 2 3 3 3 0 3
 3 3]
Purity: 0.8533333333333334


## Hasil dan Analisis

Dari keempat eksperimen di atas, dapat ditarik kesimpulan bahwa untuk dataset iris, metode *KMeans (PAM)* dapat diterapkan untuk melakukan *clustering*. Pemilihan nilai randomize_cluster=2 menghasilkan *cluster* dengan *purity* tertinggi, yaitu 0.9467. Keempat percobaan diatas menghasilkan *purity* yang berbeda meskipun nilai k sama, yaitu k=4. Hal ini dikarenakan inisiasi centroid yang random membuat hasil yang tidak sama serta pemilihan centroid mana yang akan di-random untuk setiap iterasinnya juga mempengaruhi hasil.

# Pembagian Kerja
-------------------------
Berikut adalah pembagian kerja yang dilakukan selama pelaksanaan tugas besar ini.

|      Nama     |                           Deskripsi Pekerjaan                          |
|:-------------:|:----------------------------------------------------------------------:|
| Diki Ardian   | Implementasi, eksperimen, dan analisis metode Kmeans dan Kmedoids      |
| Irfan Ariq    | Implementasi, eksperimen, dan analisis metode DBSCAN                   |
| Pratamamia AP | Implementasi, eksperimen, dan analisis metode Agglomerative Clustering |